<img src="Bilder/ost_logo.png" width="240"  align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/U05/CNN_MNIST_SOLUTION_pl.ipynb)

# Faltungsnetzwerke

Diese Übungsserie zeigt, wie man die sogenannte PyTorch Lightning API zum einfachen Aufbau von Convolutional Neural Networks in PyTorch verwendet. PyTorch Lightning ist eine High-Level-API für PyTorch, die den Trainingsprozess vereinfacht und strukturiert.

Dieses Serie zeigt auch, wie man PyTorch Lightning benutzt, um ein Modell zu speichern und zu laden, und wie man die Gewichte und Ausgaben von Faltungsschichten erhält. PyTorch Lightning wird in Zukunft sehr wahrscheinlich die Standard-API für PyTorch sein. PyTorch Lightning ist bereits sehr gut und wird ständig verbessert. Es wird also empfohlen, PyTorch Lightning zu verwenden.



## Flowchart

Das folgende Diagramm zeigt grob, wie die Daten in dem unten implementierten neuronalen Faltungsnetzwerk fliessen. 

Es gibt zwei Faltungsschichten, jeweils gefolgt von einem Down-Sampling mit Max-Pooling (in diesem Flussdiagramm nicht dargestellt). Dann folgen zwei vollständig verbundene Schichten, die in einem Softmax-Klassifikator enden.


![Flowchart](Bilder/02_network_flowchart.png)

# a) Notebook vorbereiten und Daten Laden
In diesem Code werden verschiedene Schritte durchgeführt
1. Importieren von Bibliotheken
2. Definition der Transformationen für den MNIST-Datensatz
3. Laden des MNIST-Datensatzes
4. Aufteilen des Trainingssatzes in Trainings- und Validierungssätze
5. Erstellen der DataLoader
6. Hilfsfunktion zur Visualisierung einiger Trainingsbilder

In [ ]:
# Imports und Hilfsfunktionen
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torchmetrics import Accuracy
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Transformationsdefinition für MNIST
transform = transforms.Compose(
    [
        transforms.ToTensor(),  # Konvertierung in Tensor
        transforms.Normalize((0.1307,), (0.3081,)),  # Normalisierung
    ]
)

# Laden des MNIST-Datensatzes
mnist_full = datasets.MNIST(root="data", train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(
    root="data", train=False, download=True, transform=transform
)

# Aufteilen des Trainingssatzes in Training und Validierung
train_size = int(0.9 * len(mnist_full))
val_size = len(mnist_full) - train_size
mnist_train, mnist_val = random_split(mnist_full, [train_size, val_size])

# Erstellen der DataLoader
batch_size = 128
train_loader = DataLoader(
    mnist_train, batch_size=batch_size, shuffle=True, num_workers=2
)
val_loader = DataLoader(
    mnist_val,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
    persistent_workers=True,
)
test_loader = DataLoader(
    mnist_test,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
    persistent_workers=True,
)


# Hilfsfunktion zum Visualisieren einiger MNIST-Bilder
def plot_mnist_samples(dataset, num_samples=6):
    fig, axes = plt.subplots(1, num_samples, figsize=(num_samples * 2, 2))
    for i in range(num_samples):
        image, label = dataset[i]
        axes[i].imshow(image.squeeze(), cmap="gray")
        axes[i].set_title(str(label))
        axes[i].axis("off")
    plt.show()


# Beispiel: Anzeigen einiger Trainingsbilder
plot_mnist_samples(mnist_train)


# b) Aufbau des CNN-Modells (Variante 1) mit PyTorch Lightning
Dieser Code definiert eine Convolutional Neural Network (CNN) Klasse `LitCNN` mit Konstruktor, Forward Methode Training Step, Validation Step und Test Step. Es wird auch ein Optimierer definiert.

- Netzwerkarchitektur:
    - 2 Convolutional Layers mit Max-Pooling, um Merkmale aus den Bildern zu extrahieren.
	- 2 Fully Connected Layers, um die Klassifikation durchzuführen.
- Trainingsprozess:
	- Loss: Cross-Entropy-Loss zur Klassifikation.
	- Optimizer: Adam-Optimizer zur Gewichtsaktualisierung.
	- Metrik: Genauigkeit (Accuracy) zur Bewertung.
- LightningModule-Funktionen:
	- ``training_step()``: Berechnet Loss & Accuracy für das Training.
	- ``validation_step()``: Berechnet Loss & Accuracy für die Validierung.
	- ``test_step()``: Testet das Modell auf neuen Daten.
	- ``configure_optimizers()``: Definiert den Adam-Optimizer.

Das Modell trainiert ein CNN für eine 10-Klassen-Klassifikation mit automatisiertem Training & Logging über PyTorch Lightning. 

In [ ]:
class LitCNN(pl.LightningModule):
    def __init__(self, learning_rate=1e-3):
        super(LitCNN, self).__init__()
        # Convolutional Layer 1: Input (1,28,28) -> Output (16,28,28)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(2)  # Output (16,14,14)
        # Convolutional Layer 2: Output (36,14,14)
        self.conv2 = nn.Conv2d(
            in_channels=16, out_channels=36, kernel_size=3, padding=1
        )
        self.pool2 = nn.MaxPool2d(2)  # Output (36,7,7)
        # Fully Connected Layers
        self.fc1 = nn.Linear(36 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

        self.learning_rate = learning_rate  # Lernrate
        self.accuracy = Accuracy(
            task="multiclass", num_classes=10
        )  # weil MNIST 10 Klassen hat

    def forward(self, x):
        # x: [batch, 1, 28, 28]
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)  # -> [batch, 16, 14, 14]
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)  # -> [batch, 36, 7, 7]
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)  # Cross-Entropy-Loss
        preds = torch.argmax(logits, dim=1)  # Vorhersage
        acc = self.accuracy(preds, y)  # Genauigkeit
        self.log("train_loss", loss)  # Logging
        self.log("train_acc", acc, prog_bar=True)  # Logging
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = self.accuracy(preds, y)
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = self.accuracy(preds, y)
        self.log("test_loss", loss)
        self.log("test_acc", acc)
        return {"preds": preds, "targets": y}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer


In [ ]:
from torchsummary import summary

if torch.cuda.is_available():
    device = "cuda" # GPU   
else:
    device = "cpu" # CPU

model = LitCNN().to(device)
summary(model, (1, 28, 28))


# c) Training des Modells (Variante 1)
Hier wird das Modell ``LitCNN`` mit PyTorch Lightning trainiert und die Ergebnisse als CSV-Logs gespeichert.
- Modellinstanzierung: ``LitCNN`` wird mit einer Lernrate von $0.001$ erstellt.
- Logging: CSVLogger speichert Trainingsmetriken in ``logs/mnist_model/``.
- Trainer-Erstellung:
    - Maximal 5 Epochen ``(max_epochs=5)``
    - Automatische Hardware-Auswahl ``(accelerator="auto")``
    - Ein Gerät nutzen (devices=1)
    - Training starten: ``trainer.fit()`` trainiert das Modell mit ``train_loader`` & validiert es mit ``val_loader``.

In [ ]:
from pytorch_lightning.loggers import CSVLogger

# Instanziiere das Modell und trainiere es
model = LitCNN(learning_rate=1e-3)

# Erstelle den CSVLogger
csv_logger = CSVLogger("logs", name="mnist_model")

# Erstelle den Trainer (bei Bedarf automatisch die passende Hardware auswählen)
trainer = pl.Trainer(max_epochs=5, accelerator="auto", devices=1, logger=csv_logger)
trainer.fit(model, train_loader, val_loader)


# d) Evaluation des Modells

In [ ]:
# Teste das Modell auf dem Testset
test_results = trainer.test(model, test_loader)
print("Test-Ergebnisse:", test_results)


# e) Vorhersage und Konfusionsmatrix

In [ ]:
# Berechne Vorhersagen und erstelle die Konfusionsmatrix
all_preds = []
all_targets = []

model.eval()
with torch.no_grad():
    for batch in test_loader:
        x, y = batch
        logits = model(x)
        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(y.cpu().numpy())

cm = confusion_matrix(all_targets, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title("Konfusionsmatrix - Testdaten")
plt.show()


# f) Visualisierung falsch klassifizierter Bilder

In [ ]:
def plot_example_errors(model, dataloader, num_errors=6):
    model.eval()
    error_images = []
    error_preds = []
    error_targets = []

    with torch.no_grad():
        for batch in dataloader:

            x, y = batch
            x, y = x.to(device), y.to(device)  # Move batch to GPU

            logits = model(x)
            preds = torch.argmax(logits, dim=1) 
            # Indizes der falsch klassifizierten Bilder
            incorrect = preds != y
            if incorrect.sum() > 0:
                error_images.append(x[incorrect].cpu())
                error_preds.append(preds[incorrect].cpu())
                error_targets.append(y[incorrect].cpu())
            # Beenden, wenn genügend Fehler gesammelt wurden
            if sum([img.shape[0] for img in error_images]) >= num_errors:
                break

    if error_images:
        error_images = torch.cat(error_images, dim=0)[:num_errors]
        error_preds = torch.cat(error_preds, dim=0)[:num_errors]
        error_targets = torch.cat(error_targets, dim=0)[:num_errors]
        fig, axes = plt.subplots(1, num_errors, figsize=(num_errors * 2, 2))
        for i in range(num_errors):
            axes[i].imshow(error_images[i].squeeze(), cmap="gray")
            axes[i].set_title(
                f"pred: {error_preds[i].item()}\ntrue: {error_targets[i].item()}"
            )
            axes[i].axis("off")
        plt.show()
    else:
        print("Keine falsch klassifizierten Beispiele gefunden.")


# Zeige einige falsch klassifizierte Bilder
plot_example_errors(model, test_loader, num_errors=6)


# g) Erstellung eines zweiten Modells (Variante 2) mit RMSprop


In [ ]:
class LitCNN_RMSprop(LitCNN):
    def configure_optimizers(self):
        optimizer = torch.optim.RMSprop(self.parameters(), lr=self.learning_rate)
        return optimizer


# Instanziiere das zweite Modell und trainiere es
model2 = LitCNN_RMSprop(learning_rate=1e-3).to(device)

summary(model2, (1, 28, 28))

# Erstelle den CSVLogger
csv_logger = CSVLogger("logs", name="mnist_model_rmsprop")

# Erstelle den Trainer (bei Bedarf automatisch die passende Hardware auswählen)
trainer = pl.Trainer(max_epochs=5, accelerator="auto", devices=1, logger=csv_logger)
trainer.fit(model2, train_loader, val_loader)

# Evaluation des zweiten Modells
test_results2 = trainer.test(model2, test_loader)
print("Test-Ergebnisse (RMSprop):", test_results2)

# Erzeuge die Konfusionsmatrix für model2
all_preds = []
all_targets = []

model2.eval()
with torch.no_grad():
    for batch in test_loader:
        x, y = batch
        logits = model2(x)
        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(y.cpu().numpy())

cm = confusion_matrix(all_targets, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title("Konfusionsmatrix - Testdaten (RMSprop)")
plt.show()


# h) Modell abspeichern und laden
Dieser Code speichert ein trainiertes Modell und lädt es später in eine neue Instanz.

In [ ]:
# Speichern des Modells (model2)
torch.save(model2.state_dict(), "model2.pth")
# Lösche model2 aus dem Speicher
del model2

# Lade das Modell in eine neue Instanz (model3)
model3 = LitCNN_RMSprop(learning_rate=1e-3)
model3.load_state_dict(torch.load("model2.pth"))
model3.eval()
print("Modell wurde erfolgreich geladen.")


# i) Visualisierung der Gewichte der Faltungskerne
- Jeder 3×3-Filter entspricht einer Kanten- oder Mustererkennung.
- Durch die Visualisierung kann man verstehen, welche Merkmale das Modell aus den Eingabebildern lernt.
- Die Filter detektieren z. B. Kanten, Texturen oder Kontraste in den Eingabedaten.
So kann überprüft werden, ob das Modell sinnvolle Merkmale aus den Bildern extrahiert!

In [ ]:
# Visualisiere die Filter des ersten Convolutional Layers
weights = model.conv1.weight.data.cpu().numpy()  # Shape: [16, 1, 3, 3]
fig, axes = plt.subplots(4, 4, figsize=(8, 8))
for i, ax in enumerate(axes.flat):
    if i < weights.shape[0]:
        ax.imshow(weights[i, 0, :, :], cmap="gray")
    ax.axis("off")
plt.suptitle("Gewichte der ersten Convolutional Layer")
plt.show()


# j) Visualisierung des Outputs einer Faltungsschicht
Dieser Code zeigt, wie ein Bild durch das CNN verarbeitet wird und welche Merkmale die zweite Convolutional Layer erkennt.
- Die Feature Maps zeigen aktivierte Bereiche, die bestimmte Muster im Bild repräsentieren.
- Helle Bereiche in den Visualisierungen bedeuten starke Aktivierungen für bestimmte Filter.
- Dies hilft bei der Analyse, ob das Modell sinnvolle Merkmale extrahiert!

In [ ]:
def get_conv2_output(model, x):
    with torch.no_grad():
        x = model.conv1(x)
        x = F.relu(x)
        x = model.pool1(x)
        x = model.conv2(x)
        x = F.relu(x)
        return x


# Wähle ein Beispielbild aus dem Testset
sample, _ = next(iter(test_loader))
conv2_output = get_conv2_output(model, sample[0:1].to(device))
num_feature_maps = conv2_output.shape[1]


In [ ]:
num_feature_maps


In [ ]:
import matplotlib.pyplot as plt
import math

num_rows, num_cols = 6, 6  # Define the grid size

fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 12))  # Adjust the figure size
axes = axes.flatten()  # Flatten to iterate easily

for i in range(min(num_feature_maps, num_rows * num_cols)):  # Avoid out-of-bounds errors
    axes[i].imshow(conv2_output[0, i, :, :].cpu().numpy(), cmap="gray")
    axes[i].axis("off")

# Hide unused subplots if there are fewer feature maps than grid spaces
for i in range(num_feature_maps, num_rows * num_cols):
    axes[i].axis("off")

plt.suptitle("Output der zweiten Convolutional Layer")
plt.show()
